In [1]:
#Importing needed libraries
import requests
import pandas as pd
from datetime import datetime
import geopandas as gpd
import fiona
import time
from json import JSONDecodeError
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from statsmodels.discrete.count_model import ZeroInflatedPoisson
from personal_lib import general_functions as gf


from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="capstone_test")

C:\Users\johnf\AppData\Roaming\Python\Python310\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
PAGE = 1000               # <-- enforce 1,000 rows per page
TIMEOUT = 30
MAX_RETRIES = 5
BACKOFF_BASE = 1.5

session = requests.Session()
headers = {}

def fetch_all_rows_1k(api_url: str, source_years: str, source_info_url: str) -> pd.DataFrame:
    offset = 0
    frames = []

    while True:
        params = {"$limit": PAGE, "$offset": offset}

        for attempt in range(1, MAX_RETRIES + 1):
            try:
                resp = session.get(api_url, params=params, headers=headers, timeout=TIMEOUT)
                if resp.status_code in (429, 502, 503, 504):
                    time.sleep(BACKOFF_BASE ** attempt * (0.1 * attempt))
                    continue
                resp.raise_for_status()

                if "json" not in resp.headers.get("Content-Type", "").lower():
                    preview = resp.text[:200]
                    raise ValueError(f"Non-JSON response (status {resp.status_code}): {preview}")

                data_chunk = resp.json()
                if not data_chunk:
                    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

                # Create DataFrame
                df = pd.DataFrame(data_chunk)

                # 🔑 Add your metadata columns here
                df["source_years"] = source_years
                df["source_api_url"] = api_url
                df["source_info_url"] = source_info_url

                frames.append(df)

                # If less than PAGE, stop; otherwise keep paginating
                if len(data_chunk) < PAGE:
                    return pd.concat(frames, ignore_index=True)

                offset += PAGE
                break  # Success, go to next page

            except (requests.RequestException, JSONDecodeError, ValueError) as e:
                if attempt == MAX_RETRIES:
                    print(f"⚠️ Failed fetching {api_url} at offset {offset}: {e}")
                    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
                time.sleep(BACKOFF_BASE ** attempt * (0.1 * attempt))


In [25]:
## There are mulitple sources of yearly data. Compiling the source URLs here as well as the API urls to grab
## privately owned buildings over 25,000 ft2 and in City-owned buildings over 10,000 ft2
building_energy_LL84_sources={
# "2022+":{
#     "api":"https://data.cityofnewyork.us/resource/5zyy-y8am.json",
#     "info":"https://data.cityofnewyork.us/Environment/NYC-Building-Energy-and-Water-Data-Disclosure-for-/5zyy-y8am/about_data"
#                               },
#                               "2021":{
#     "api":"https://data.cityofnewyork.us/resource/7x5e-2fxh.json",
#     "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/7x5e-2fxh/about_data"
#                               },
#                               "2020":{
#     "api":"https://data.cityofnewyork.us/resource/usc3-8zwd.json",
#     "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/usc3-8zwd/about_data"
#                                   },
#                               "2019":{
#     "api":"https://data.cityofnewyork.us/resource/wcm8-aq5w.json",
#     "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/wcm8-aq5w/about_data"
#                                   },
#                               "2018":{
#     "api":"https://data.cityofnewyork.us/resource/4tys-3tzj.json",
#     "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/4tys-3tzj/about_data"
#                                   },
                              "2017":{
    "api":"https://data.cityofnewyork.us/resource/4t62-jm4m.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/4t62-jm4m/about_data"
                                  },
    #                           "2016":{
    # "api":"https://data.cityofnewyork.us/resource/utpj-74fz.json",
    # "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/utpj-74fz/about_data"
    #                               },
    #                           "2015":{
    # "api":"https://data.cityofnewyork.us/resource/77q4-nkfh.json",
    # "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/77q4-nkfh/about_data"
    #                               },
    #                           "2014":{
    # "api":"https://data.cityofnewyork.us/resource/nbun-wekj.json",
    # "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/nbun-wekj/about_data"
    #                               },
    #                           "2013":{
    # "api":"https://data.cityofnewyork.us/resource/yr5p-wjer.json",
    # "info":"http://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/yr5p-wjer"
    #                               },
                              "2012":{
    "api":"https://data.cityofnewyork.us/resource/r6ub-zhff.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/r6ub-zhff/about_data"
                                  },
                              "2011":{
    "api":"https://data.cityofnewyork.us/resource/k7nh-aufb.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/k7nh-aufb/about_data"
                                  },
                              "2010":{
    "api":"https://data.cityofnewyork.us/resource/kswi-37bp.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/kswi-37bp/about_data"
                                  },
#Monthly Data? 2018-2023
    #                           "Monthly":{
    # "api":"https://data.cityofnewyork.us/resource/fvp3-gcb2.json",
    # "info":"https://data.cityofnewyork.us/Environment/Local-Law-84-Monthly-Data/fvp3-gcb2/about_data"
    # }
}


In [26]:
agg_running_list = []
for k, v in building_energy_LL84_sources.items():
    print(f"Fetching {k} -> {v['api']}")
    df = fetch_all_rows_1k(v["api"], k, v["info"])
    if not df.empty:
        agg_running_list.append(df)
    else:
        print(f"Warning: no rows returned for {k} ({v['api']}).")

Fetching 2017 -> https://data.cityofnewyork.us/resource/4t62-jm4m.json
Fetching 2012 -> https://data.cityofnewyork.us/resource/r6ub-zhff.json
Fetching 2011 -> https://data.cityofnewyork.us/resource/k7nh-aufb.json
Fetching 2010 -> https://data.cityofnewyork.us/resource/kswi-37bp.json


In [27]:
nyc_building_energy_2010_2017 = pd.concat(agg_running_list, ignore_index=True) if agg_running_list else pd.DataFrame()
print(f"Total rows: {len(nyc_building_energy_2010_2017)}")

Total rows: 70797


In [29]:
## Dropping Other Property Type Columns 
to_drop =[
    'data_center_ups_output_meter',
 'data_center_pdu_input_meter',
 'data_center_pdu_output_meter',
 'data_center_it_equipment',
 'data_center_it_site_energy',
 'data_center_it_source_energy',
 'data_center_pue',
 'data_center_national_median',
 'data_center_gross_floor_area',
 'data_center_ups_system',
 'data_center_it_energy',
 'data_center_cooling_equipment',
'supermarkets_grocery_gross',
 'supermarkets_grocery_number',
 'supermarkets_grocery_number_1',
 'supermarkets_grocery_number_2',
 'supermarkets_grocery_percent',
 'supermarkets_grocery_presence',
 'supermarkets_grocery_walk',
 'supermarkets_grocery_weekly',
 'supermarkets_grocery_workers',
'house_of_worship_gross_floor',
 'house_of_worship_pc_density',
 'house_of_worship_weekly',
 'house_of_worship_presence',
'residence_halls_dormitories',
'medical_office_percent_cooled',
 'residence_halls_dormitories_1',
 'residence_halls_dormitories_2',
 'residence_halls_dormitories_3',
 'residence_halls_dormitories_4',
'hotel_onsite_laundry_short',
 'warehouse_unrefrigerated',
 'warehouse_unrefrigerated_1',
 'warehouse_unrefrigerated_2',
 'warehouse_unrefrigerated_3',
 'warehouse_unrefrigerated_4',
 'warehouse_unrefrigerated_5',
 'warehouse_unrefrigerated_6',
 'hospital_gross_floor_area',
 'hospital_laboratory_y_1_n',
 'hospital_laundry_facility',
 'hospital_maximum_number_of',
 'hospital_number_of_buildings',
 'warehouse_refrigerated_gross',
 'warehouse_refrigerated_weekly',
 'warehouse_refrigerated_workers',
 'hospital_number_of_licensed',
 'multifamily_home_dishwashers'
## Banking oriented COlumns
'bank_branch_computer_density',
 'bank_branch_gross_floor_area',
 'bank_branch_gross_floor_area_ft',
 'bank_branch_number_of',
 'bank_branch_number_of_workers',
 'bank_branch_percent_that',
 'bank_branch_weekly_operating',
 'bank_branch_worker_density',
## College / Uni / School
'college_university_gross',
 'college_university_gross_floor_area_ft',
 'college_university_number',
    'k_12_school_computer_density',
 'k_12_school_cooking_facilities',
 'k_12_school_gross_floor_area',
 'k_12_school_gross_floor_area_ft',
 'k_12_school_high_school',
 'k_12_school_percent_that',
 'k_12_school_refrigeration',
 'k_12_school_weekend_operation',
 'laboratory_gross_floor_area_ft',
    'library_gross_floor_area',
# OTher
"automobile_dealership_gross",
'convenience_store_without',
'data_center_energy_estimates_applied',
 'data_center_gross_floor_area',
 'data_center_gross_floor_area_ft',
 'data_center_it_energy',
 'data_center_it_energy_configuration',
 'data_center_it_equipment_input_meter_kwh',
 'data_center_it_site_energy',
 'data_center_it_site_energy_kwh',
 'data_center_it_source_energy_kbtu',
 'data_center_national_median',
 'data_center_pdu_input_meter_kwh',
 'data_center_pdu_output_meter_kwh',
 'data_center_ups_output_meter_kwh',
 'enclosed_mall_gross_floor',
 'enclosed_mall_gross_floor_area_ft',
 'fast_food_restaurant_gross',
 'financial_office_gross_floor',
 'financial_office_gross_floor_area_ft',
 'financial_office_number_of',
 'financial_office_number_of_1',
 'financial_office_number_of_computers',
 'financial_office_number_of_workers_on_main_shift',
 'financial_office_weekly',
 'financial_office_weekly_operating_hours',
 'fitness_center_health_club',
 'fitness_center_health_club_gym_gross_floor_area_ft',
 'food_sales_gross_floor_area',
 'food_sales_gross_floor_area_ft',
 'food_service_gross_floor',
 'food_service_gross_floor_area_ft',
     'worship_facility_computer',
 'worship_facility_cooking',
 'worship_facility_gross_floor',
 'worship_facility_gross_floor_area_ft',
 'worship_facility_weekly',
    'supermarket_grocery_cooking',
 'supermarket_grocery_gross',
 'supermarket_grocery_gross_floor_area_ft',
 'supermarket_grocery_number',
 'supermarket_grocery_number_1',
 'supermarket_grocery_number_2',
 'supermarket_grocery_number_of_open_or_closed_refrigeration_freezer_units',
 'supermarket_grocery_number_of_walk_in_refrigeration_freezer_units',
 'supermarket_grocery_percent',
 'supermarket_grocery_walk',
 'supermarket_grocery_weekly',
 'supermarket_grocery_worker',
 'swimming_pool_approximate',
 'swimming_pool_location_of',
 'swimming_pool_months_in_use',
     'social_meeting_hall_gross',
 'social_meeting_hall_gross_floor_area_ft',
     'residence_hall_dormitory',
 'residence_hall_dormitory_1',
 'residence_hall_dormitory_2',
 'residence_hall_dormitory_3',
 'residence_hall_dormitory_4',
 'residence_hall_dormitory_gross_floor_area_ft',
 'restaurant_gross_floor_area',
 'restaurant_gross_floor_area_ft',
 'restaurant_weekly_operating',
 'restaurant_weekly_operating_hours',
 'restaurant_worker_density',
 'restaurant_worker_density_number_per_1_000_sq_ft',
 'retail_store_cash_register',
 'retail_store_computer_density',
 'retail_store_exterior_entrance',
 'retail_store_gross_floor',
 'retail_store_gross_floor_area_ft',
 'retail_store_number_of_open',
 'retail_store_number_of_open_or_closed_refrigeration_freezer_units',
 'retail_store_number_of_walk',
 'retail_store_number_of_walk_in_refrigeration_freezer_units',
 'retail_store_open_or_closed',
 'retail_store_percent_that',
 'retail_store_walk_in',
 'retail_store_weekly_operating',
 'retail_store_worker_density',
 'self_storage_facility_gross',
 'self_storage_facility_gross_floor_area_ft',
 'senior_care_community_average',
 'senior_care_community_gross',
 'senior_care_community_maximum',
 'senior_care_community_number',
 'senior_care_community_number_1',
 'senior_care_community_number_2',
 'senior_care_community_number_3',
 'senior_care_community_number_4',
 'senior_care_community_number_5',
 'senior_care_community_number_6',
 'senior_care_community_percent',
 'senior_living_community_gross_floor_area_ft',
 'senior_living_community_living_unit_density_number_per_1_000_sq_ft',
    'movie_theater_gross_floor_area_ft',
    'non_refrigerated_warehouse',
 'non_refrigerated_warehouse_1',
 'non_refrigerated_warehouse_2',
 'non_refrigerated_warehouse_3',
 'non_refrigerated_warehouse_4',
 'non_refrigerated_warehouse_5',
 'non_refrigerated_warehouse_gross_floor_area_ft',
     'parking_completely_enclosed',
 'parking_completely_enclosed_parking_garage_size_ft',
 'parking_gross_floor_area',
 'parking_gross_floor_area_ft',
 'parking_open_parking_lot',
 'parking_open_parking_lot_size_ft',
 'parking_partially_enclosed',
 'parking_partially_enclosed_parking_garage_size_ft',
     'strip_mall_gross_floor_area',
## Hostpial / Hotel
    'hospital_general_medical',
 'hospital_general_medical_1',
 'hospital_general_medical_10',
 'hospital_general_medical_11',
 'hospital_general_medical_12',
 'hospital_general_medical_13',
 'hospital_general_medical_14',
 'hospital_general_medical_15',
 'hospital_general_medical_16',
 'hospital_general_medical_17',
 'hospital_general_medical_2',
 'hospital_general_medical_3',
 'hospital_general_medical_4',
 'hospital_general_medical_5',
 'hospital_general_medical_6',
 'hospital_general_medical_7',
 'hospital_general_medical_8',
 'hospital_general_medical_9',
 'hotel_amount_of_laundry',
 'hotel_cooking_facilities',
 'hotel_full_service_spa_floor',
 'hotel_gross_floor_area_ft',
 'hotel_gym_fitness_center',
 'hotel_gym_fitness_center_floor_area_ft',
 'hotel_number_of_rooms',
 'hotel_percent_that_can_be',
 'hotel_room_density_number',
 'hotel_type_of_laundry_facility',
 'hotel_worker_density_number',
 'urgent_care_clinic_other',
 'urgent_care_clinic_other_outpatient_gross_floor_area_ft',
    'mailing_center_post_office_gross_floor_area_ft',
 'manufacturing_industrial_plant_gross_floor_area_ft',
 'medical_office_gross_floor',
 'medical_office_gross_floor_area_ft',
 'medical_office_mri_machine',
 'medical_office_number_of',
 'medical_office_number_of_1',
 'medical_office_number_of_computers',
 'medical_office_number_of_mri_machines',
 'medical_office_number_of_workers_on_main_shift',
 'medical_office_percent_that',
 'medical_office_percent_that_1',
 'medical_office_percent_that_can_be_cooled',
 'medical_office_percent_that_can_be_heated',
 'medical_office_weekly',
 'medical_office_weekly_operating_hours',
    'museum_gross_floor_area_ft',
    'office_computer_density_number',
 'office_gross_floor_area_ft',
 'office_number_of_computers',
 'office_number_of_workers',
 'office_number_of_workers_on_main_shift',
 'office_percent_that_can_be',
 'office_percent_that_can_be_1',
 'office_percent_that_can_be_cooled',
 'office_percent_that_can_be_heated',
 'office_weekly_operating_hours',
 'office_worker_density_number',
 'office_worker_density_number_per_1_000_sq_ft',
     'adult_education_gross_floor',
 'adult_education_gross_floor_area_ft',
]
nyc_building_energy_2010_2017 = nyc_building_energy_2010_2017.drop(columns=[i for i in to_drop if i in nyc_building_energy_2010_2017.columns ])

In [33]:
### Columns to Drop because potentially irrelevant or redundant for 
nyc_building_energy_processing = nyc_building_energy_2010_2017.copy()
nyc_building_energy_processing["year_ending"] = pd.to_datetime(nyc_building_energy_processing["year_ending"], errors="coerce")
# Optional: extract just the year if that’s all you need
nyc_building_energy_processing["year_ending_year"] = nyc_building_energy_processing["year_ending"].dt.year
nyc_building_energy_processing["year_agg"] = nyc_building_energy_processing["year_ending_year"].combine_first(nyc_building_energy_processing["source_years"])


In [35]:

nyc_building_energy_processing[~nyc_building_energy_processing["property_id"].isnull()].groupby(["year_agg"]).agg({"city":"count"})

,city
year_agg,
2012.0,13382
2017.0,34686


In [36]:

nyc_building_energy_processing[nyc_building_energy_processing["property_id"].isnull()].groupby(["year_agg"]).agg({"city":"count"})

,city
year_agg,
2010,10329
2011,12400


In [37]:
nyc_building_energy_processing.groupby(["year_agg"]).agg({"city":"count"})

,city
year_agg,
2012.0,13382
2017.0,34686
2010,10329
2011,12400


In [38]:
nyc_building_energy_processing[~nyc_building_energy_processing["latitude"].isnull()].groupby(["year_agg"]).agg({"city":"count"})

,city
year_agg,
2012.0,12485
2017.0,33436
2010,9572
2011,11399


In [12]:


## Fruther Limiting to those that are under 10 Stories. 
nyc_building_energy_processing = nyc_building_energy_processing.replace("Not Available",np.nan).dropna(how='all',axis=1)
nyc_building_energy_processing = nyc_building_energy_processing.dropna(how='all',axis=1)

In [20]:
 nyc_building_energy_processing.groupby(["year_agg"]).agg({'address_1':'count'})

,address_1
year_agg,
2017.0,34686
2010,10186


In [24]:
test = nyc_building_energy_processing[nyc_building_energy_processing['year_agg']=='2010'].dropna(how='all',axis=1)
test[test['latitude'].isnull()]

,address_1,city,postcode,weather_normalized_site_eui,difference_from_national,district_steam_use_kbtu,natural_gas_use_kbtu,borough,latitude,longitude,community_board,council_district,census_tract,nta,fuel_oil_2_use_kbtu,fuel_oil_4_use_kbtu,other_weekly_operating_hours,county,address_2,diesel_2_use_kbtu,source_years,source_api_url,source_info_url,building_id,facility_name,period_ending_date,rating,state,total_floor_space_sq_ft,facility_type,last_modified_date,eligibility_for_energy_star,energy_star_application_status,current_rating_1_100,target_rating_1_100,current_indirect_ghg_emissions,current_total_ghg_emissions,current_energy_period_ending,current_water_period_ending,current_site_energy_intensity,current_total_site_energy,national_average_site_eui,target_site_energy_intensity,electricity_use_kbtu,electricity_use_kwh,current_source_energy,current_total_source_energy,current_weather_normalized,national_average_source_eui,bank_financial_institution,bank_financial_institution_1,bank_financial_institution_2,bank_financial_institution_3,bank_financial_institution_4,bank_financial_institution_5,office_gross_floor_area_sq,office_office_air_conditioned,office_pc_density,office_workers_density,release_date,bin,bbl,current_direct_ghg_emissions,natural_gas_use_therms,fuel_oil_56_use_kbtu,multifamily_home_gross_floor,multifamily_home_number_of,other_space_type_name,other_gross_floor_area_sq,other_number_of_pcs,other_workers_on_main_shift,indoor_water_use_kgal,indoor_water_use_per_sq_ft,total_indoor_and_outdoor,multifamily_home_laundry,multifamily_home_laundry_1,multifamily_home_number_of_1,multifamily_home_percent,multifamily_home_percent_1,multifamily_home_total_number,retail_gross_floor_area_sq,retail_cash_register_density,retail_exterior_entrance,retail_workers_density,retail_weekly_operating_hours,retail_walk_in_refrig_density,retail_percent_cooled,retail_pc_density,retail_open_closed_refrig,retail_number_of_walk_in,retail_number_of_open_or,multifamily_home_market_rate,hotel_gross_floor_area_sq,hotel_percent_cooled,hotel_presence_of_cooking,parking_enclosed_floor_area,parking_non_enclosed_floor,parking_open_floor_area_w,parking_weekly_hours_of_access,k_12_school_high_school_y,k_12_school_open_weekends,k_12_school_pc_density,k_12_school_percent_cooled,k_12_school_walk_in_refrig,k_12_school_presence_of,year_s_labeled,national_average_pue,data_center_annual_it_energy,swimming_pool_size,swimming_pool_indoor_outdoor,total_green_power_purchased,current_site_ups_output_energy,current_source_it_energy,building_profile_status,total_avoided_ghg_emissions,current_site_pdu_input_energy,current_total_on_site,total_on_site_electric,percent_of_electricity_from,year_agg
34696,NaN,Arlington,22201,40.8,-8.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARLINGTON,NaN,NaN,2010,https://data.cityofnewyork.us/resource/kswi-37...,https://data.cityofnewyork.us/Environment/Ener...,2637200,Sample Facility,2010-12-31T00:00:00.000,57,Virginia,15000,Office,2011-03-04T00:00:00.000,Not Eligible: Rating must be 75 or above,No Status Available,57,0,91.64,91.64,2010-12-31T00:00:00.000,2200-01-01T00:00:00.000,40.9,612907.8,44.5,0.0,612907.80,179633.0,136.5,2047112.0,136.3,148.6,NaN,NaN,NaN,NaN,NaN,NaN,15000,25.00,2.13,2.33,2011-04-15T09:09:03.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010
34698,NaN,Arlington,22201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARLINGTON,NaN,NaN,2010,https://data.cityofnewyork.us/resource/kswi-37...,https://data.cityofnewyork.us/Environment/Ener...,2800482,Sample Facility,2010-12-31T00:00:00.000,0,Virginia,15000,Office,2011-07-18T00:00:00.000,Not Eligible: Rating must be 75 or above,No Status Available,57,NaN,NaN,NaN,2011-04-30T00:00:00.000,2200-01-01T00:00:00.000,NaN,NaN,44.5,NaN,NaN,NaN,NaN,NaN,NaN,148.6,NaN,NaN,NaN,NaN,NaN,NaN,

In [20]:
nyc_building_energy_processing[nyc_building_energy_processing['property_id'].isnull()].groupby(["year_ending_year"]).agg({'address_1':'count'})
# nyc_building_energy_processing[["property_id","custom_property_id_1_id"]].drop_duplicates()

,address_1
year_ending_year,


In [ ]:
## Beginning of Limiting to Multifamily Homes (Step 1)
residential = nyc_building_energy_processing[(
    (nyc_building_energy_processing["list_of_all_property_use"].isin([i for i in nyc_building_energy_processing["list_of_all_property_use"].unique() if 'Multifamily Housing' in str(i)]))
    |(nyc_building_energy_processing["list_of_all_property_use"].isnull() 
      & (nyc_building_energy_processing["primary_property_type_self"]=="Multifamily Housing")
      & ((nyc_building_energy_processing["primary_property_type"]=="Multifamily Housing")|
         (nyc_building_energy_processing["primary_property_type"].isnull()))))]

C:\Users\johnf\AppData\Local\Temp\ipykernel_57712\1164641468.py:54: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  nyc_building_energy_processing = nyc_building_energy_processing.replace("Not Available",np.nan).dropna(how='all',axis=1)


KeyError: 'primary_property_type'